In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

## Data Preparation

In [50]:
from sklearn.decomposition import PCA
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import RandomizedSearchCV, train_test_split;
import numpy;
import pandas;
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier;
import sys;
from keras.utils import to_categorical;
REMOTE_MODULE_PATH = '/home/macbookretina/automatic-music-genre-classification/feature_extraction_deep_learning'
# LOCAL_MODULE_PATH = '/Users/macbookretina/Desktop/automatic-music-genre-classification/feature_extraction_deep_learning'
sys.path.insert(1, REMOTE_MODULE_PATH)
from custom_module.utilities import *

In [51]:
features = pandas.read_csv(MOUNTED_DATASET_PATH + '/data/cleaned_features.csv', index_col=0)
features

,genre_label,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,3,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,3,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [52]:
y = features.loc[:,'genre_label']
X = features.loc[:, features.columns.difference(['genre_label'])]
X

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,mean_db4_cD3,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,-0.000069,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,0.000984,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,0.000019,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,-0.001993,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,0.000076,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,-0.000065,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,-0.000009,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,-0.000024,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,-0.000132,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [53]:
y.value_counts(normalize=True)

3    0.333434
1    0.333434
2    0.333131
Name: genre_label, dtype: float64

In [54]:
predictors_all_label = X.columns.values
predictors_with_outliers_label = ['mean_beats', 'mean_spec_bw', 'mean_spec_centroid', 'mean_spec_rolloff',
                          'median_beats','median_spec_bw', 'median_spec_centroid', 
                          'median_spec_rolloff', 'std_beats', 'std_mfcc_1', 'var_spec_centroid',
                          'var_spec_rolloff'
                         ]
predictors_without_outliers_label = X.loc[
    :, X.columns.difference(predictors_with_outliers_label)].columns.values

In [55]:
preprocess_sp = standardization_pipeline(
    predictors_all_label,
    predictors_with_outliers_label,
    predictors_without_outliers_label)

preprocess_np = normalization_pipeline(
    predictors_all_label)

### Split & Transform Data

In [56]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True)
X_train.shape

(2639, 217)

In [57]:
pipeline_estimator_sp = standardization_pipeline(
    predictors_all_label,
    predictors_with_outliers_label,
    predictors_without_outliers_label
).fit(X_train)
# X_train_sp = pipeline_estimator_sp.transform(X_train)
X_test_sp = pipeline_estimator_sp.transform(X_test)
X_test_sp.shape

(660, 217, 1)

In [58]:
pipeline_estimator_np = normalization_pipeline(
    predictors_all_label).fit(X_train)
# X_train_np = pipeline_estimator_np.transform(X_train)
X_test_np = pipeline_estimator_np.transform(X_test)
X_test_np.shape

(660, 217, 1)

In [59]:
_, ncols, _ = X_test_np.shape

In [60]:
# one hot encoding our labels
y_train = to_categorical(y_train, 4)
y_test = to_categorical(y_test, 4)

In [61]:
y_train.shape, y_test.shape

((2639, 4), (660, 4))

## Predictive Modelling

In [13]:
early_stopping_cb = callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)

In [14]:
param_dist = {
    'clf__n_hidden': [0,1,2,3,4,5],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100)
}

### FNN With Standardized Input

In [15]:
# wrap the function with keras wrapper
fnn_1 = KerasClassifier(build_fn=set_shape_create_model('fnn', ncols))

In [16]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', fnn_1)
])

In [17]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [18]:
# %%timeit
# fit model
search_1 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 217)               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 872       
Total params: 872
Trainable params: 872
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 0s 8ms/step - loss: 1.4425 - accuracy: 0.4360 - val_accuracy: 0.5652 - val_loss: 1.1254
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0695 - accuracy: 0.5918 - val_accuracy: 0.6379 - val_loss: 0.9823
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9488 - accuracy: 0.6418 - val_accuracy: 0.6485 - val_loss: 0.9161
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8781 - accuracy: 0.6725 

55/55 [==============================] - 0s 2ms/step - loss: 0.7354 - accuracy: 0.7061 - val_accuracy: 0.6833 - val_loss: 0.8010
Epoch 9/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7198 - accuracy: 0.7169 - val_accuracy: 0.6879 - val_loss: 0.7953
Epoch 10/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7081 - accuracy: 0.7209 - val_accuracy: 0.6894 - val_loss: 0.7875
Epoch 11/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6978 - accuracy: 0.7186 - val_accuracy: 0.6970 - val_loss: 0.7770
Epoch 12/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6901 - accuracy: 0.7203 - val_accuracy: 0.7000 - val_loss: 0.7674
Epoch 13/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6806 - accuracy: 0.7311 - val_accuracy: 0.7091 - val_loss: 0.7675
Epoch 14/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6726 - accuracy: 0.7311 - val_accuracy: 0.7106 - val_loss: 0.7626
Epoch 15/100
55/55 [

Epoch 62/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5878 - accuracy: 0.7665 - val_accuracy: 0.7136 - val_loss: 0.7035
Epoch 63/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5879 - accuracy: 0.7608 - val_accuracy: 0.7197 - val_loss: 0.7051
Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5867 - accuracy: 0.7614 - val_accuracy: 0.7152 - val_loss: 0.7038
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5869 - accuracy: 0.7602 - val_accuracy: 0.7136 - val_loss: 0.7023
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5864 - accuracy: 0.7648 - val_accuracy: 0.7136 - val_loss: 0.7037
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5864 - accuracy: 0.7602 - val_accuracy: 0.7121 - val_loss: 0.7002
Epoch 68/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5855 - accuracy: 0.7688 - val_accuracy: 0.7121 - val_loss: 0.7023
Epoch 

Epoch 87/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5779 - accuracy: 0.7686 - val_accuracy: 0.7242 - val_loss: 0.7146
Epoch 88/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5773 - accuracy: 0.7663 - val_accuracy: 0.7258 - val_loss: 0.7144
Epoch 89/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5767 - accuracy: 0.7686 - val_accuracy: 0.7242 - val_loss: 0.7143
Epoch 90/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5762 - accuracy: 0.7692 - val_accuracy: 0.7258 - val_loss: 0.7142
Epoch 91/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5755 - accuracy: 0.7681 - val_accuracy: 0.7273 - val_loss: 0.7141
Epoch 92/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5750 - accuracy: 0.7681 - val_accuracy: 0.7273 - val_loss: 0.7139
Epoch 93/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5744 - accuracy: 0.7681 - val_accuracy: 0.7242 - val_loss: 0.7138
Epoch 

Epoch 96/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5793 - accuracy: 0.7658 - val_accuracy: 0.7318 - val_loss: 0.6947
Epoch 97/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5787 - accuracy: 0.7652 - val_accuracy: 0.7303 - val_loss: 0.6946
Epoch 98/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5782 - accuracy: 0.7641 - val_accuracy: 0.7318 - val_loss: 0.6945
Epoch 99/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5777 - accuracy: 0.7646 - val_accuracy: 0.7288 - val_loss: 0.6942
Epoch 100/100
55/55 [==============================] - 0s 1ms/step - loss: 0.5755 - accuracy: 0.7652
Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 217)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 83)               

...

Epoch 86/100
55/55 [==============================] - 0s 3ms/step - loss: 0.5767 - accuracy: 0.7595 - val_accuracy: 0.7212 - val_loss: 0.7115
Epoch 87/100
55/55 [==============================] - 0s 3ms/step - loss: 0.5776 - accuracy: 0.7516 - val_accuracy: 0.7212 - val_loss: 0.7146
Epoch 88/100
55/55 [==============================] - 0s 3ms/step - loss: 0.5769 - accuracy: 0.7521 - val_accuracy: 0.7121 - val_loss: 0.7184
Epoch 89/100
55/55 [==============================] - 0s 3ms/step - loss: 0.5754 - accuracy: 0.7499 - val_accuracy: 0.7197 - val_loss: 0.7209
Epoch 90/100
55/55 [==============================] - 0s 3ms/step - loss: 0.5772 - accuracy: 0.7544 - val_accuracy: 0.7121 - val_loss: 0.7142
Epoch 91/100
55/55 [==============================] - 0s 3ms/step - loss: 0.5731 - accuracy: 0.7561 - val_accuracy: 0.7152 - val_loss: 0.7182
Epoch 92/100
55/55 [==============================] - 0s 3ms/step - loss: 0.5778 - accuracy: 0.7527 - val_accuracy: 0.7136 - val_loss: 0.7145
Epoch 

Epoch 100/100
55/55 [==============================] - 0s 1ms/step - loss: 0.5560 - accuracy: 0.7784
Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_27 (Flatten)         (None, 217)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 89)                19402     
_________________________________________________________________
dense_76 (Dense)             (None, 89)                8010      
_________________________________________________________________
dense_77 (Dense)             (None, 89)                8010      
_________________________________________________________________
dense_78 (Dense)             (None, 4)                 360       
Total params: 35,782
Trainable params: 35,782
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [

Epoch 1/100
55/55 [==============================] - 0s 4ms/step - loss: 1.0623 - accuracy: 0.5543 - val_accuracy: 0.6530 - val_loss: 0.8994
Epoch 2/100
55/55 [==============================] - 0s 3ms/step - loss: 0.8404 - accuracy: 0.6555 - val_accuracy: 0.6773 - val_loss: 0.8225
Epoch 3/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7757 - accuracy: 0.6799 - val_accuracy: 0.6879 - val_loss: 0.7898
Epoch 4/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7388 - accuracy: 0.6981 - val_accuracy: 0.6924 - val_loss: 0.7727
Epoch 5/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7121 - accuracy: 0.7027 - val_accuracy: 0.7000 - val_loss: 0.7579
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6915 - accuracy: 0.7095 - val_accuracy: 0.6985 - val_loss: 0.7506
Epoch 7/100
55/55 [==============================] - 0s 3ms/step - loss: 0.6754 - accuracy: 0.7163 - val_accuracy: 0.7076 - val_loss: 0.7414
Epoch 8/100
5

Epoch 59/100
55/55 [==============================] - 0s 3ms/step - loss: 0.4513 - accuracy: 0.8238 - val_accuracy: 0.7455 - val_loss: 0.6719
Epoch 60/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4492 - accuracy: 0.8260 - val_accuracy: 0.7470 - val_loss: 0.6714
Epoch 61/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4471 - accuracy: 0.8277 - val_accuracy: 0.7455 - val_loss: 0.6710
Epoch 62/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4449 - accuracy: 0.8260 - val_accuracy: 0.7485 - val_loss: 0.6712
Epoch 63/100
55/55 [==============================] - 0s 3ms/step - loss: 0.4430 - accuracy: 0.8289 - val_accuracy: 0.7470 - val_loss: 0.6711
Epoch 64/100
55/55 [==============================] - 0s 3ms/step - loss: 0.4409 - accuracy: 0.8300 - val_accuracy: 0.7500 - val_loss: 0.6706
Epoch 65/100
55/55 [==============================] - 0s 3ms/step - loss: 0.4390 - accuracy: 0.8300 - val_accuracy: 0.7530 - val_loss: 0.6697
Epoch 

Epoch 78/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4128 - accuracy: 0.8494 - val_accuracy: 0.7136 - val_loss: 0.6812
Epoch 79/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4113 - accuracy: 0.8540 - val_accuracy: 0.7152 - val_loss: 0.6814
Epoch 80/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4096 - accuracy: 0.8528 - val_accuracy: 0.7121 - val_loss: 0.6812
Epoch 81/100
55/55 [==============================] - 0s 3ms/step - loss: 0.4080 - accuracy: 0.8551 - val_accuracy: 0.7121 - val_loss: 0.6806
Epoch 82/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4063 - accuracy: 0.8568 - val_accuracy: 0.7136 - val_loss: 0.6808
Epoch 83/100
55/55 [==============================] - 0s 2ms/step - loss: 0.4047 - accuracy: 0.8534 - val_accuracy: 0.7106 - val_loss: 0.6811
Epoch 84/100
55/55 [==============================] - 0s 3ms/step - loss: 0.4031 - accuracy: 0.8568 - val_accuracy: 0.7121 - val_loss: 0.6812
Epoch 

In [19]:
search_1.best_score_

0.7267904363151932

In [20]:
search_1.best_params_

{'clf__activation': 'relu',
 'clf__kernel_initializer': 'normal',
 'clf__n_hidden': 2,
 'clf__n_neurons': 90,
 'clf__optimizer': 'adam'}

In [67]:
search_1.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_with_outliers', Pipeline(memory=None,
     st..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f5814472748>)])

### FNN With Normalized Input

In [22]:
# wrap the function with keras wrapper
fnn_2 = KerasClassifier(set_shape_create_model('fnn_np', ncols))

In [23]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', fnn_2)
])

In [24]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [25]:
# %%timeit
# fit model
search_2 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 217)               0         
_________________________________________________________________
dense_90 (Dense)             (None, 4)                 872       
Total params: 872
Trainable params: 872
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 1s 9ms/step - loss: 1.1641 - accuracy: 0.4105 - val_accuracy: 0.5227 - val_loss: 1.0545
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0281 - accuracy: 0.5151 - val_accuracy: 0.5576 - val_loss: 0.9961
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9699 - accuracy: 0.5907 - val_accuracy: 0.6030 - val_loss: 0.9550
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9308 - accuracy: 0.61

55/55 [==============================] - 0s 2ms/step - loss: 0.8785 - accuracy: 0.6299 - val_accuracy: 0.6000 - val_loss: 0.8911
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8607 - accuracy: 0.6367 - val_accuracy: 0.6424 - val_loss: 0.8630
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8457 - accuracy: 0.6453 - val_accuracy: 0.6500 - val_loss: 0.8487
Epoch 9/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8339 - accuracy: 0.6441 - val_accuracy: 0.6636 - val_loss: 0.8366
Epoch 10/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8236 - accuracy: 0.6470 - val_accuracy: 0.6621 - val_loss: 0.8286
Epoch 11/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8150 - accuracy: 0.6578 - val_accuracy: 0.6364 - val_loss: 0.8366
Epoch 12/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8084 - accuracy: 0.6549 - val_accuracy: 0.6818 - val_loss: 0.8082
Epoch 13/100
55/55 [==

Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6997 - accuracy: 0.6970 - val_accuracy: 0.7152 - val_loss: 0.7252
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7010 - accuracy: 0.7015 - val_accuracy: 0.7136 - val_loss: 0.7199
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7005 - accuracy: 0.6981 - val_accuracy: 0.6985 - val_loss: 0.7232
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6989 - accuracy: 0.7044 - val_accuracy: 0.6970 - val_loss: 0.7253
Epoch 68/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6984 - accuracy: 0.7044 - val_accuracy: 0.7000 - val_loss: 0.7208
Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6976 - accuracy: 0.6976 - val_accuracy: 0.7152 - val_loss: 0.7194
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6980 - accuracy: 0.6953 - val_accuracy: 0.7091 - val_loss: 0.7165
Epoch 

55/55 [==============================] - 0s 2ms/step - loss: 1.0796 - accuracy: 0.4315 - val_accuracy: 0.4621 - val_loss: 1.0756
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0596 - accuracy: 0.4912 - val_accuracy: 0.4924 - val_loss: 1.0626
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0445 - accuracy: 0.5264 - val_accuracy: 0.5030 - val_loss: 1.0477
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0323 - accuracy: 0.5600 - val_accuracy: 0.5152 - val_loss: 1.0398
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0218 - accuracy: 0.5605 - val_accuracy: 0.5364 - val_loss: 1.0244
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0113 - accuracy: 0.5850 - val_accuracy: 0.5758 - val_loss: 1.0124
Epoch 9/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0021 - accuracy: 0.5918 - val_accuracy: 0.5515 - val_loss: 1.0092
Epoch 10/100
55/55 [=====

Epoch 61/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8329 - accuracy: 0.6487 - val_accuracy: 0.6697 - val_loss: 0.8436
Epoch 62/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8317 - accuracy: 0.6498 - val_accuracy: 0.6682 - val_loss: 0.8426
Epoch 63/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8304 - accuracy: 0.6492 - val_accuracy: 0.6697 - val_loss: 0.8414
Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8291 - accuracy: 0.6526 - val_accuracy: 0.6682 - val_loss: 0.8418
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8278 - accuracy: 0.6492 - val_accuracy: 0.6682 - val_loss: 0.8407
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8266 - accuracy: 0.6498 - val_accuracy: 0.6682 - val_loss: 0.8397
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8250 - accuracy: 0.6492 - val_accuracy: 0.6682 - val_loss: 0.8363
Epoch 

...

Epoch 1/100
55/55 [==============================] - 0s 4ms/step - loss: 1.3101 - accuracy: 0.3479 - val_accuracy: 0.3409 - val_loss: 1.2061
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 1.1547 - accuracy: 0.3348 - val_accuracy: 0.3409 - val_loss: 1.1300
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 1.1163 - accuracy: 0.3371 - val_accuracy: 0.3167 - val_loss: 1.1128
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 1.1069 - accuracy: 0.3275 - val_accuracy: 0.3167 - val_loss: 1.1070
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 1.1032 - accuracy: 0.3331 - val_accuracy: 0.3167 - val_loss: 1.1047
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 1.1015 - accuracy: 0.3303 - val_accuracy: 0.3167 - val_loss: 1.1045
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 1.1004 - accuracy: 0.3326 - val_accuracy: 0.3167 - val_loss: 1.1022
Epoch 8/100
5

Epoch 59/100
55/55 [==============================] - 0s 3ms/step - loss: 0.9748 - accuracy: 0.5748 - val_accuracy: 0.5682 - val_loss: 0.9768
Epoch 60/100
55/55 [==============================] - 0s 3ms/step - loss: 0.9691 - accuracy: 0.5668 - val_accuracy: 0.5879 - val_loss: 0.9692
Epoch 61/100
55/55 [==============================] - 0s 3ms/step - loss: 0.9629 - accuracy: 0.5782 - val_accuracy: 0.5909 - val_loss: 0.9635
Epoch 62/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9567 - accuracy: 0.5787 - val_accuracy: 0.5727 - val_loss: 0.9593
Epoch 63/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9506 - accuracy: 0.5702 - val_accuracy: 0.5818 - val_loss: 0.9517
Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9435 - accuracy: 0.5867 - val_accuracy: 0.5879 - val_loss: 0.9451
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9376 - accuracy: 0.5833 - val_accuracy: 0.5909 - val_loss: 0.9384
Epoch 

Epoch 65/100
55/55 [==============================] - 0s 3ms/step - loss: 0.8947 - accuracy: 0.5915 - val_accuracy: 0.6121 - val_loss: 0.9013
Epoch 66/100
55/55 [==============================] - 0s 3ms/step - loss: 0.8900 - accuracy: 0.6074 - val_accuracy: 0.5970 - val_loss: 0.8975
Epoch 67/100
55/55 [==============================] - 0s 3ms/step - loss: 0.8860 - accuracy: 0.6057 - val_accuracy: 0.6045 - val_loss: 0.8946
Epoch 68/100
55/55 [==============================] - 0s 3ms/step - loss: 0.8817 - accuracy: 0.6080 - val_accuracy: 0.6030 - val_loss: 0.8899
Epoch 69/100
55/55 [==============================] - 0s 3ms/step - loss: 0.8779 - accuracy: 0.6051 - val_accuracy: 0.6076 - val_loss: 0.8871
Epoch 70/100
55/55 [==============================] - 0s 3ms/step - loss: 0.8743 - accuracy: 0.6028 - val_accuracy: 0.6136 - val_loss: 0.8830
Epoch 71/100
55/55 [==============================] - 0s 3ms/step - loss: 0.8708 - accuracy: 0.6085 - val_accuracy: 0.6106 - val_loss: 0.8823
Epoch 

55/55 [==============================] - 0s 3ms/step - loss: 0.6545 - accuracy: 0.7129 - val_accuracy: 0.7333 - val_loss: 0.6919
Epoch 27/100
55/55 [==============================] - 0s 3ms/step - loss: 0.6468 - accuracy: 0.7180 - val_accuracy: 0.7333 - val_loss: 0.6737
Epoch 28/100
55/55 [==============================] - 0s 3ms/step - loss: 0.6443 - accuracy: 0.7146 - val_accuracy: 0.7348 - val_loss: 0.6671
Epoch 29/100
55/55 [==============================] - 0s 3ms/step - loss: 0.6319 - accuracy: 0.7362 - val_accuracy: 0.7076 - val_loss: 0.7349
Epoch 30/100
55/55 [==============================] - 0s 3ms/step - loss: 0.6363 - accuracy: 0.7283 - val_accuracy: 0.6303 - val_loss: 0.8071
Epoch 31/100
55/55 [==============================] - 0s 3ms/step - loss: 0.6223 - accuracy: 0.7271 - val_accuracy: 0.5879 - val_loss: 0.9581
Epoch 32/100
55/55 [==============================] - 0s 3ms/step - loss: 0.6336 - accuracy: 0.7220 - val_accuracy: 0.7242 - val_loss: 0.6734
Epoch 33/100
55/55 

83/83 [==============================] - 0s 3ms/step - loss: 0.5959 - accuracy: 0.7491 - val_accuracy: 0.7455 - val_loss: 0.6411
Epoch 31/100
83/83 [==============================] - 0s 3ms/step - loss: 0.6085 - accuracy: 0.7404 - val_accuracy: 0.7318 - val_loss: 0.6693
Epoch 32/100
83/83 [==============================] - ETA: 0s - loss: 0.6025 - accuracy: 0.74 - 0s 2ms/step - loss: 0.5904 - accuracy: 0.7510 - val_accuracy: 0.7106 - val_loss: 0.6946
Epoch 33/100
83/83 [==============================] - 0s 2ms/step - loss: 0.5856 - accuracy: 0.7518 - val_accuracy: 0.7273 - val_loss: 0.6619
Epoch 34/100
83/83 [==============================] - 0s 2ms/step - loss: 0.5964 - accuracy: 0.7473 - val_accuracy: 0.7621 - val_loss: 0.6434
Epoch 35/100
83/83 [==============================] - 0s 2ms/step - loss: 0.6076 - accuracy: 0.7317 - val_accuracy: 0.7364 - val_loss: 0.6731
Epoch 36/100
83/83 [==============================] - 0s 2ms/step - loss: 0.5927 - accuracy: 0.7529 - val_accuracy: 0.7

In [26]:
search_2.best_score_

0.7226222125180249

In [70]:
search_2.best_params_

{'clf__activation': 'elu',
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__n_hidden': 3,
 'clf__n_neurons': 77,
 'clf__optimizer': 'adam'}

In [73]:
search_2.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None,
     steps=[('col..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f58144e4dd8>)])

### CNN With Standardized Input

In [28]:
param_dist = {
    'clf__n_hidden': [1, 2, 3],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100),
    'clf__filters': [16, 32, 64],
#     'clf__kernel_size': [3,4,5],
    'clf__dropout':[.1, .15, .2, .25]
}

In [29]:
# wrap the function with keras wrapper
cnn_1 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn', ncols))

In [30]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', cnn_1)
])

In [31]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [32]:
# %%timeit
# fit model
search_3 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 215, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 107, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 105, 64)           6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 52, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 96)            18528     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 25, 96)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 23, 128)           36992   

Epoch 5/100
55/55 [==============================] - 1s 15ms/step - loss: 0.8728 - accuracy: 0.6117 - val_accuracy: 0.6348 - val_loss: 0.8360
Epoch 6/100
55/55 [==============================] - 1s 15ms/step - loss: 0.8655 - accuracy: 0.6157 - val_accuracy: 0.6394 - val_loss: 0.8289
Epoch 7/100
55/55 [==============================] - 1s 15ms/step - loss: 0.8419 - accuracy: 0.6146 - val_accuracy: 0.6470 - val_loss: 0.8156
Epoch 8/100
55/55 [==============================] - 1s 15ms/step - loss: 0.8434 - accuracy: 0.6259 - val_accuracy: 0.6348 - val_loss: 0.8106
Epoch 9/100
55/55 [==============================] - 1s 15ms/step - loss: 0.8302 - accuracy: 0.6231 - val_accuracy: 0.6485 - val_loss: 0.8075
Epoch 10/100
55/55 [==============================] - 1s 15ms/step - loss: 0.8192 - accuracy: 0.6379 - val_accuracy: 0.6530 - val_loss: 0.8096
Epoch 11/100
55/55 [==============================] - 1s 15ms/step - loss: 0.8138 - accuracy: 0.6316 - val_accuracy: 0.6576 - val_loss: 0.8015
Epoc

Epoch 13/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7742 - accuracy: 0.6727 - val_accuracy: 0.6742 - val_loss: 0.7810
Epoch 14/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7650 - accuracy: 0.6676 - val_accuracy: 0.6667 - val_loss: 0.7751
Epoch 15/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7563 - accuracy: 0.6784 - val_accuracy: 0.6682 - val_loss: 0.7732
Epoch 16/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7474 - accuracy: 0.6801 - val_accuracy: 0.6788 - val_loss: 0.7697
Epoch 17/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7391 - accuracy: 0.6898 - val_accuracy: 0.6848 - val_loss: 0.7616
Epoch 18/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7417 - accuracy: 0.6852 - val_accuracy: 0.6833 - val_loss: 0.7619
Epoch 19/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7240 - accuracy: 0.6801 - val_accuracy: 0.6848 - val_loss: 0.7611

Epoch 1/100
55/55 [==============================] - 1s 18ms/step - loss: 0.9783 - accuracy: 0.5549 - val_accuracy: 0.6227 - val_loss: 0.8649
Epoch 2/100
55/55 [==============================] - 1s 15ms/step - loss: 0.8181 - accuracy: 0.6464 - val_accuracy: 0.6621 - val_loss: 0.7640
Epoch 3/100
55/55 [==============================] - 1s 15ms/step - loss: 0.6832 - accuracy: 0.7032 - val_accuracy: 0.6894 - val_loss: 0.8361
Epoch 4/100
55/55 [==============================] - 1s 15ms/step - loss: 0.5938 - accuracy: 0.7550 - val_accuracy: 0.6788 - val_loss: 0.7832
Epoch 5/100
55/55 [==============================] - 1s 15ms/step - loss: 0.5006 - accuracy: 0.7925 - val_accuracy: 0.7212 - val_loss: 0.7338
Epoch 6/100
55/55 [==============================] - 1s 15ms/step - loss: 0.4077 - accuracy: 0.8397 - val_accuracy: 0.7076 - val_loss: 0.8454
Epoch 7/100
55/55 [==============================] - 1s 15ms/step - loss: 0.3185 - accuracy: 0.8829 - val_accuracy: 0.7000 - val_loss: 0.9133
Epoch 

55/55 [==============================] - 1s 14ms/step - loss: 0.8880 - accuracy: 0.6089 - val_accuracy: 0.6439 - val_loss: 0.7950
Epoch 2/100
55/55 [==============================] - 1s 12ms/step - loss: 0.7049 - accuracy: 0.6981 - val_accuracy: 0.6939 - val_loss: 0.7414
Epoch 3/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6458 - accuracy: 0.7345 - val_accuracy: 0.6985 - val_loss: 0.7400
Epoch 4/100
55/55 [==============================] - 1s 12ms/step - loss: 0.5778 - accuracy: 0.7658 - val_accuracy: 0.7000 - val_loss: 0.7714
Epoch 5/100
55/55 [==============================] - 1s 12ms/step - loss: 0.5351 - accuracy: 0.7789 - val_accuracy: 0.6939 - val_loss: 0.7536
Epoch 6/100
55/55 [==============================] - 1s 12ms/step - loss: 0.4608 - accuracy: 0.8067 - val_accuracy: 0.7015 - val_loss: 0.7836
Epoch 7/100
55/55 [==============================] - 1s 12ms/step - loss: 0.3906 - accuracy: 0.8488 - val_accuracy: 0.6818 - val_loss: 0.8982
Epoch 8/100
55/55 

...

55/55 [==============================] - 0s 8ms/step - loss: 0.5436 - accuracy: 0.7675 - val_accuracy: 0.7121 - val_loss: 0.6909
Epoch 48/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5330 - accuracy: 0.7783 - val_accuracy: 0.7152 - val_loss: 0.6922
Epoch 49/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5379 - accuracy: 0.7777 - val_accuracy: 0.7182 - val_loss: 0.6937
Epoch 50/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5380 - accuracy: 0.7754 - val_accuracy: 0.7152 - val_loss: 0.6916
Epoch 51/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5356 - accuracy: 0.7754 - val_accuracy: 0.7106 - val_loss: 0.6917
Epoch 52/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5226 - accuracy: 0.7874 - val_accuracy: 0.7167 - val_loss: 0.6927
Epoch 53/100
55/55 [==============================] - 0s 8ms/step - loss: 0.5237 - accuracy: 0.7840 - val_accuracy: 0.7182 - val_loss: 0.6917
Epoch 54/100
55/55 

Epoch 27/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5798 - accuracy: 0.7619 - val_accuracy: 0.6955 - val_loss: 0.7126
Epoch 28/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5707 - accuracy: 0.7676 - val_accuracy: 0.6939 - val_loss: 0.7129
Epoch 29/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5811 - accuracy: 0.7602 - val_accuracy: 0.6909 - val_loss: 0.7104
Epoch 30/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5752 - accuracy: 0.7733 - val_accuracy: 0.6909 - val_loss: 0.7153
Epoch 31/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5766 - accuracy: 0.7619 - val_accuracy: 0.6939 - val_loss: 0.7089
Epoch 32/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5764 - accuracy: 0.7716 - val_accuracy: 0.6939 - val_loss: 0.7089
Epoch 33/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5606 - accuracy: 0.7727 - val_accuracy: 0.6909 - val_loss: 0.7102
Epoch 

55/55 [==============================] - 1s 14ms/step - loss: 0.5180 - accuracy: 0.7879 - val_accuracy: 0.7227 - val_loss: 0.6728
Epoch 29/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5093 - accuracy: 0.7948 - val_accuracy: 0.7212 - val_loss: 0.6777
Epoch 30/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5125 - accuracy: 0.7868 - val_accuracy: 0.7242 - val_loss: 0.6655
Epoch 31/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5084 - accuracy: 0.7862 - val_accuracy: 0.7258 - val_loss: 0.6678
Epoch 32/100
55/55 [==============================] - 1s 14ms/step - loss: 0.4983 - accuracy: 0.8010 - val_accuracy: 0.7288 - val_loss: 0.6651
Epoch 33/100
55/55 [==============================] - 1s 14ms/step - loss: 0.4935 - accuracy: 0.8050 - val_accuracy: 0.7212 - val_loss: 0.6788
Epoch 34/100
55/55 [==============================] - 1s 14ms/step - loss: 0.4973 - accuracy: 0.7965 - val_accuracy: 0.7258 - val_loss: 0.6752
Epoch 35/100

55/55 [==============================] - 1s 11ms/step - loss: 0.4097 - accuracy: 0.8449 - val_accuracy: 0.7258 - val_loss: 0.6699
Epoch 66/100
55/55 [==============================] - 1s 11ms/step - loss: 0.4137 - accuracy: 0.8500 - val_accuracy: 0.7212 - val_loss: 0.6707
Epoch 67/100
55/55 [==============================] - 0s 4ms/step - loss: 0.4061 - accuracy: 0.8455
Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_75 (Conv1D)           (None, 215, 16)           64        
_________________________________________________________________
max_pooling1d_75 (MaxPooling (None, 107, 16)           0         
_________________________________________________________________
conv1d_76 (Conv1D)           (None, 105, 32)           1568      
_________________________________________________________________
max_pooling1d_76 (MaxPooling (None, 52, 32)            0         
____________________

Epoch 2/100
83/83 [==============================] - 1s 11ms/step - loss: 0.7544 - accuracy: 0.6700 - val_accuracy: 0.6924 - val_loss: 0.7473
Epoch 3/100
83/83 [==============================] - 1s 11ms/step - loss: 0.7025 - accuracy: 0.6950 - val_accuracy: 0.6894 - val_loss: 0.7248
Epoch 4/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6867 - accuracy: 0.7094 - val_accuracy: 0.6848 - val_loss: 0.7440
Epoch 5/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6714 - accuracy: 0.7215 - val_accuracy: 0.6985 - val_loss: 0.7233
Epoch 6/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6616 - accuracy: 0.7249 - val_accuracy: 0.7015 - val_loss: 0.7069
Epoch 7/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6475 - accuracy: 0.7257 - val_accuracy: 0.7061 - val_loss: 0.7066
Epoch 8/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6343 - accuracy: 0.7389 - val_accuracy: 0.7136 - val_loss: 0.7008
Epoch 

In [76]:
search_3.best_score_

0.7275483285265015

In [77]:
search_3.best_params_

{'clf__activation': 'relu',
 'clf__dropout': 0.15,
 'clf__filters': 64,
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__n_hidden': 1,
 'clf__n_neurons': 77,
 'clf__optimizer': 'adagrad'}

In [78]:
search_3.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_with_outliers', Pipeline(memory=None,
     st..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f57c4523a58>)])

### CNN With Normalized Input

In [35]:
# wrap the function with keras wrapper
cnn_2 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn_np', ncols))

In [36]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', cnn_1)
])

In [37]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [38]:
# %%timeit
# fit model
search_4 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_89 (Conv1D)           (None, 215, 32)           128       
_________________________________________________________________
max_pooling1d_89 (MaxPooling (None, 107, 32)           0         
_________________________________________________________________
conv1d_90 (Conv1D)           (None, 105, 64)           6208      
_________________________________________________________________
max_pooling1d_90 (MaxPooling (None, 52, 64)            0         
_________________________________________________________________
conv1d_91 (Conv1D)           (None, 50, 96)            18528     
_________________________________________________________________
max_pooling1d_91 (MaxPooling (None, 25, 96)            0         
_________________________________________________________________
conv1d_92 (Conv1D)           (None, 23, 128)           36992   

Epoch 8/100
55/55 [==============================] - 1s 14ms/step - loss: 0.9606 - accuracy: 0.5378 - val_accuracy: 0.5591 - val_loss: 0.9278
Epoch 9/100
55/55 [==============================] - 1s 14ms/step - loss: 0.9589 - accuracy: 0.5350 - val_accuracy: 0.5606 - val_loss: 0.9249
Epoch 10/100
55/55 [==============================] - 1s 15ms/step - loss: 0.9453 - accuracy: 0.5355 - val_accuracy: 0.5591 - val_loss: 0.9200
Epoch 11/100
55/55 [==============================] - 1s 15ms/step - loss: 0.9412 - accuracy: 0.5406 - val_accuracy: 0.5636 - val_loss: 0.9132
Epoch 12/100
55/55 [==============================] - 1s 15ms/step - loss: 0.9395 - accuracy: 0.5452 - val_accuracy: 0.5848 - val_loss: 0.9041
Epoch 13/100
55/55 [==============================] - 1s 14ms/step - loss: 0.9262 - accuracy: 0.5532 - val_accuracy: 0.5864 - val_loss: 0.9006
Epoch 14/100
55/55 [==============================] - 1s 14ms/step - loss: 0.9223 - accuracy: 0.5753 - val_accuracy: 0.5909 - val_loss: 0.8957
E

Epoch 16/100
55/55 [==============================] - 1s 14ms/step - loss: 0.8878 - accuracy: 0.5824 - val_accuracy: 0.6121 - val_loss: 0.8719
Epoch 17/100
55/55 [==============================] - 1s 14ms/step - loss: 0.8769 - accuracy: 0.6006 - val_accuracy: 0.6091 - val_loss: 0.8628
Epoch 18/100
55/55 [==============================] - 1s 16ms/step - loss: 0.8787 - accuracy: 0.5972 - val_accuracy: 0.6182 - val_loss: 0.8597
Epoch 19/100
55/55 [==============================] - 1s 16ms/step - loss: 0.8585 - accuracy: 0.6034 - val_accuracy: 0.6152 - val_loss: 0.8546
Epoch 20/100
55/55 [==============================] - 1s 17ms/step - loss: 0.8699 - accuracy: 0.6114 - val_accuracy: 0.6212 - val_loss: 0.8505
Epoch 21/100
55/55 [==============================] - 1s 16ms/step - loss: 0.8534 - accuracy: 0.6023 - val_accuracy: 0.6091 - val_loss: 0.8478
Epoch 22/100
55/55 [==============================] - 1s 17ms/step - loss: 0.8482 - accuracy: 0.6131 - val_accuracy: 0.6364 - val_loss: 0.8424

Epoch 12/100
55/55 [==============================] - 1s 15ms/step - loss: 0.4220 - accuracy: 0.8289 - val_accuracy: 0.6758 - val_loss: 0.7851
Epoch 13/100
55/55 [==============================] - 1s 15ms/step - loss: 0.3788 - accuracy: 0.8545 - val_accuracy: 0.6379 - val_loss: 1.0467
Epoch 14/100
55/55 [==============================] - 1s 15ms/step - loss: 0.3532 - accuracy: 0.8590 - val_accuracy: 0.6591 - val_loss: 0.9989
Epoch 15/100
55/55 [==============================] - 0s 5ms/step - loss: 0.5544 - accuracy: 0.7760
Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_105 (Conv1D)          (None, 215, 32)           128       
_________________________________________________________________
max_pooling1d_105 (MaxPoolin (None, 107, 32)           0         
_________________________________________________________________
conv1d_106 (Conv1D)          (None, 105, 64)           6208   

Epoch 1/100
55/55 [==============================] - 1s 17ms/step - loss: 1.0804 - accuracy: 0.4545 - val_accuracy: 0.6076 - val_loss: 0.8435
Epoch 2/100
55/55 [==============================] - 1s 14ms/step - loss: 0.8312 - accuracy: 0.6193 - val_accuracy: 0.6652 - val_loss: 0.7671
Epoch 3/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7588 - accuracy: 0.6710 - val_accuracy: 0.5864 - val_loss: 0.8914
Epoch 4/100
55/55 [==============================] - 1s 14ms/step - loss: 0.7156 - accuracy: 0.6881 - val_accuracy: 0.6591 - val_loss: 0.7791
Epoch 5/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6508 - accuracy: 0.7244 - val_accuracy: 0.6439 - val_loss: 0.7882
Epoch 6/100
55/55 [==============================] - 1s 14ms/step - loss: 0.6325 - accuracy: 0.7443 - val_accuracy: 0.7197 - val_loss: 0.6911
Epoch 7/100
55/55 [==============================] - 1s 14ms/step - loss: 0.5962 - accuracy: 0.7670 - val_accuracy: 0.6667 - val_loss: 0.7575
Epoch 

...

55/55 [==============================] - 1s 12ms/step - loss: 0.6854 - accuracy: 0.7055 - val_accuracy: 0.6833 - val_loss: 0.7318
Epoch 83/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6926 - accuracy: 0.7112 - val_accuracy: 0.6955 - val_loss: 0.7253
Epoch 84/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6997 - accuracy: 0.6998 - val_accuracy: 0.6909 - val_loss: 0.7260
Epoch 85/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6904 - accuracy: 0.7004 - val_accuracy: 0.6864 - val_loss: 0.7244
Epoch 86/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6911 - accuracy: 0.7021 - val_accuracy: 0.6879 - val_loss: 0.7295
Epoch 87/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6947 - accuracy: 0.7055 - val_accuracy: 0.6818 - val_loss: 0.7306
Epoch 88/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6907 - accuracy: 0.6936 - val_accuracy: 0.6773 - val_loss: 0.7323
Epoch 89/100

55/55 [==============================] - 1s 12ms/step - loss: 0.7187 - accuracy: 0.6856 - val_accuracy: 0.6970 - val_loss: 0.7269
Epoch 86/100
55/55 [==============================] - 1s 12ms/step - loss: 0.7180 - accuracy: 0.6839 - val_accuracy: 0.7015 - val_loss: 0.7279
Epoch 87/100
55/55 [==============================] - 1s 12ms/step - loss: 0.7212 - accuracy: 0.6879 - val_accuracy: 0.6909 - val_loss: 0.7296
Epoch 88/100
55/55 [==============================] - 1s 12ms/step - loss: 0.7206 - accuracy: 0.6873 - val_accuracy: 0.6909 - val_loss: 0.7294
Epoch 89/100
55/55 [==============================] - 1s 12ms/step - loss: 0.7151 - accuracy: 0.6896 - val_accuracy: 0.7030 - val_loss: 0.7254
Epoch 90/100
55/55 [==============================] - 1s 12ms/step - loss: 0.7158 - accuracy: 0.6896 - val_accuracy: 0.7015 - val_loss: 0.7257
Epoch 91/100
55/55 [==============================] - 1s 12ms/step - loss: 0.7119 - accuracy: 0.6822 - val_accuracy: 0.7015 - val_loss: 0.7238
Epoch 92/100

55/55 [==============================] - 1s 11ms/step - loss: 0.6993 - accuracy: 0.7051 - val_accuracy: 0.7015 - val_loss: 0.7218
Epoch 89/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7049 - accuracy: 0.6977 - val_accuracy: 0.7030 - val_loss: 0.7213
Epoch 90/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7026 - accuracy: 0.6926 - val_accuracy: 0.6909 - val_loss: 0.7250
Epoch 91/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7029 - accuracy: 0.7045 - val_accuracy: 0.6864 - val_loss: 0.7251
Epoch 92/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6965 - accuracy: 0.7011 - val_accuracy: 0.7076 - val_loss: 0.7196
Epoch 93/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6963 - accuracy: 0.6949 - val_accuracy: 0.7061 - val_loss: 0.7174
Epoch 94/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6986 - accuracy: 0.7000 - val_accuracy: 0.7000 - val_loss: 0.7181
Epoch 95/100

Epoch 1/100
55/55 [==============================] - 1s 14ms/step - loss: 1.1515 - accuracy: 0.3479 - val_accuracy: 0.4106 - val_loss: 1.0378
Epoch 2/100
55/55 [==============================] - 1s 11ms/step - loss: 0.9207 - accuracy: 0.5406 - val_accuracy: 0.6091 - val_loss: 0.8313
Epoch 3/100
55/55 [==============================] - 1s 11ms/step - loss: 0.8151 - accuracy: 0.6134 - val_accuracy: 0.6621 - val_loss: 0.7774
Epoch 4/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7605 - accuracy: 0.6418 - val_accuracy: 0.6894 - val_loss: 0.7262
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7057 - accuracy: 0.6890 - val_accuracy: 0.7227 - val_loss: 0.6958
Epoch 6/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6891 - accuracy: 0.7038 - val_accuracy: 0.6818 - val_loss: 0.7211
Epoch 7/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6303 - accuracy: 0.7339 - val_accuracy: 0.6970 - val_loss: 0.7029
Epoch 

Epoch 1/100
83/83 [==============================] - 1s 13ms/step - loss: 0.9905 - accuracy: 0.5032 - val_accuracy: 0.6515 - val_loss: 0.8241
Epoch 2/100
83/83 [==============================] - 1s 11ms/step - loss: 0.8217 - accuracy: 0.6264 - val_accuracy: 0.6939 - val_loss: 0.7892
Epoch 3/100
83/83 [==============================] - 1s 11ms/step - loss: 0.7528 - accuracy: 0.6711 - val_accuracy: 0.6909 - val_loss: 0.7478
Epoch 4/100
83/83 [==============================] - 1s 11ms/step - loss: 0.7400 - accuracy: 0.6828 - val_accuracy: 0.6864 - val_loss: 0.7135
Epoch 5/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6950 - accuracy: 0.6927 - val_accuracy: 0.7197 - val_loss: 0.6734
Epoch 6/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6713 - accuracy: 0.7139 - val_accuracy: 0.7030 - val_loss: 0.7017
Epoch 7/100
83/83 [==============================] - 1s 11ms/step - loss: 0.6518 - accuracy: 0.7272 - val_accuracy: 0.7242 - val_loss: 0.6626
Epoch 

In [79]:
search_4.best_score_

0.7347480240713786

In [80]:
search_4.best_params_

{'clf__activation': 'relu',
 'clf__dropout': 0.15,
 'clf__filters': 64,
 'clf__kernel_initializer': 'normal',
 'clf__n_hidden': 1,
 'clf__n_neurons': 79,
 'clf__optimizer': 'adam'}

In [81]:
search_4.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None,
     steps=[('col..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f57c487ca90>)])

In [69]:
# np.random.seed(23456)

# # Initiating an empty neural network
# cnn_model_2 = Sequential(name='cnn_2')

# # Adding convolutional layer
# cnn_model_2.add(layers.Conv1D(filters=16,
#                      kernel_size=(3),
#                      activation='relu',
#                      input_shape=(ncols,1)))

# # Adding max pooling layer
# cnn_model_2.add(layers.MaxPooling1D(pool_size=(2)))

# # Adding convolutional layer
# cnn_model_2.add(layers.Conv1D(filters=32,
#                      kernel_size=(3),
#                      activation='relu'))

# # Adding max pooling layer
# cnn_model_2.add(layers.MaxPooling1D(pool_size=(2)))

# # Adding a flattened layer to input our image data
# cnn_model_2.add(layers.Flatten())

# # Adding a dense layer with 64 neurons
# cnn_model_2.add(layers.Dense(64, activation='relu'))

# # Adding a dropout layer for regularization
# cnn_model_2.add(layers.Dropout(0.25))

# # Adding an output layer
# cnn_model_2.add(layers.Dense(4, activation='softmax'))

# # Compiling our neural network
# cnn_model_2.compile(loss='categorical_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])

# # Fitting our neural network
# history = cnn_model_2.fit(X_train_np,
#                         y_train, 
#                         batch_size=16,
#                         validation_data=(X_test_np, y_test),
#                         epochs=15)

Epoch 1/15
165/165 [==============================] - 1s 6ms/step - loss: 1.0459 - accuracy: 0.4498 - val_accuracy: 0.5848 - val_loss: 0.8830
Epoch 2/15
165/165 [==============================] - 1s 6ms/step - loss: 0.8354 - accuracy: 0.6158 - val_accuracy: 0.6258 - val_loss: 0.7989
Epoch 3/15
165/165 [==============================] - 1s 5ms/step - loss: 0.7803 - accuracy: 0.6586 - val_accuracy: 0.6485 - val_loss: 0.7730
Epoch 4/15
165/165 [==============================] - 1s 5ms/step - loss: 0.7515 - accuracy: 0.6669 - val_accuracy: 0.6742 - val_loss: 0.7459
Epoch 5/15
165/165 [==============================] - 1s 6ms/step - loss: 0.7214 - accuracy: 0.6866 - val_accuracy: 0.6652 - val_loss: 0.7491
Epoch 6/15
165/165 [==============================] - 1s 5ms/step - loss: 0.6948 - accuracy: 0.7067 - val_accuracy: 0.6894 - val_loss: 0.7141
Epoch 7/15
165/165 [==============================] - 1s 5ms/step - loss: 0.6921 - accuracy: 0.7090 - val_accuracy: 0.6712 - val_loss: 0.7402
Epoch 